In [16]:
pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import pandas as pd
import requests 
import threading

In [18]:
# Load the data
tickets = pd.read_csv('/content/drive/MyDrive/challenge_dataset.csv',  parse_dates=True)

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
tickets_reduced= tickets[["origin","destination","flight_num"]]

In [21]:
tickets_reduced=tickets_reduced.drop_duplicates(subset="flight_num", keep= "last")

In [22]:
ciudades_dest = [x for x in tickets["destination"]]
ciudades_origen = [x for x in tickets["origin"]]
ciudades = ciudades_dest + ciudades_origen
ciudades = list( dict.fromkeys(ciudades) )




In [23]:
def temp_de_ciudades(ciudades_i, ciudades_latlon):
  for ciudad in ciudades_i:
    if ciudad in ciudades_origen:
      lat=tickets[tickets["origin"] == ciudad ]["origin_latitude"].iloc[0]
      lon= tickets[tickets["origin"] == ciudad ]["origin_longitude"].iloc[0] 
    else:
      lat = tickets[tickets["destination"] == ciudad ]["destination_latitude"].iloc[0]
      lon= tickets[tickets["destination"] == ciudad ]["destination_longitude"].iloc[0]
  
    
    url="http://api.openweathermap.org/data/2.5/forecast?" + \
      "lat=" + str(lat)+ "&lon=" + str(lon)+"&cnt=5" + \
      "&appid=" + "d1a4fab2d46987ad898458389baa88a2"

    try:
      response= requests.get(url)
    except Exception as e:
      print(f"Error getting temp for url:{url}.\n Skipping")


    weather_prev = response.json()

    # La API da la temperatura cada 3 horas, asi que el 5to elemento son 12 PM
    weather = weather_prev["list"][4]


    temp = weather["main"]["temp"]
    ciudades_latlon[ciudad] = (temp)








In [24]:
ciudades_temps = {}
ciudades_latlon = {}
num_threads = 4
paso = len(ciudades) // num_threads
modulo = len(ciudades) % num_threads

# Si la division deja un residuo, agregaos un thread extra
if modulo > 0:
  num_threads += 1
threads = [None] * num_threads


print(f'Haciendo las llamadas al api de openwather en {num_threads} procesos, '
  f'con {paso} ciudades por proceso')



for i in range(num_threads):
    if i == num_threads - 1 and modulo > 0:
      ciudades_i = ciudades[i*paso:(i*paso) + modulo]
    else:
      ciudades_i = ciudades[i*paso:(i+1)*paso]

    
    # print(ciudades_temp)
    threads[i] = threading.Thread(target=temp_de_ciudades,
                                  args=(ciudades_i,
                                        ciudades_latlon))
    threads[i].start()

for i in range(num_threads):
    threads[i].join()


ciudades_latlon


Haciendo las llamadas al api de openwather en 4 procesos, con 13 ciudades por proceso


{'MTY': 301.19,
 'OAX': 294.07,
 'ZCL': 291.62,
 'MZT': 301.64,
 'TLC': 288.76,
 'HUX': 301.51,
 'AMS': 298.72,
 'GUA': 293.65,
 'MEX': 290.64,
 'ZIH': 301.98,
 'ATL': 294.97,
 'VSA': 301.77,
 'TAM': 301.29,
 'PVR': 301.52,
 'CEN': 301.6,
 'BZE': 300.27,
 'GDL': 292.49,
 'LIM': 289.64,
 'MAD': 307.84,
 'DFW': 296.85,
 'CJS': 296.96,
 'HAV': 303.38,
 'SCL': 289.19,
 'ORD': 296.3,
 'CUN': 303.96,
 'BOG': 288.75,
 'AGU': 293.08,
 'PHX': 303.53,
 'TIJ': 295.22,
 'MIA': 303.85,
 'CZM': 302.74,
 'PHL': 300.14,
 'HMO': 301.15,
 'LAX': 293.3,
 'CUU': 296.4,
 'CLT': 297.06,
 'CME': 300.51,
 'JFK': 297.14,
 'QRO': 291.75,
 'YYZ': 296.63,
 'MID': 303.65,
 'TRC': 299.33,
 'BJX': 292.08,
 'IAH': 301.93,
 'CTM': 301.87,
 'PXM': 301.99,
 'PBC': 290.65,
 'YVR': 291.78,
 'VER': 300.4,
 'ACA': 298.06,
 'SLP': 292.12,
 'CDG': 300.63}

In [26]:
temp_origin = [ciudades_latlon[x] for x in tickets_reduced["origin"] ]
temp_destiny=[ciudades_latlon[x] for x in tickets_reduced["destination"] ]

tickets_reduced["origin_weather"]=temp_origin
tickets_reduced['destination_weather']=temp_destiny 
tickets_reduced

,origin,destination,flight_num,origin_weather,destination_weather
1,MTY,TLC,119,301.19,288.76
2,MEX,MTY,2100,290.64,301.19
3,MTY,MEX,2101,301.19,290.64
9,MEX,MTY,2102,290.64,301.19
10,MTY,MEX,2103,301.19,290.64
...,...,...,...,...,...
2915,GDL,TIJ,110,292.49,295.22
2916,MEX,GDL,1104,290.64,292.49
2918,MEX,GDL,1120,290.64,292.49
2919,GDL,MEX,1127,292.49,290.64
